# Xarray Fundamentals

**Attribution**: This notebook is a revision of the `xarray` Fundamentals [notebook](https://earth-env-data-science.github.io/lectures/xarray/xarray.html) by Ryan Abernathy from [An Introduction to Earth and Environmental Data Science](https://earth-env-data-science.github.io/intro.html).

In this lecture, we will have a deep dive to `xarray` package. So far in this course, we have used `rioxarray` for raster data processing, and `stackstac` for analyzing stacks of satellite imagery. Both of these packages are built on top of `xarray`; however, we haven't use all the functionalities of `xarray` yet. 

In the lecture on Raster Data Processing, we introduced `Dataset` in `rioxarray` and discussed how it is different from `DataArray`. Here we will have a complete review of data structures in `xarray` and learn how to create a `DataArray` or `Dataset` from scratch.

You can access this notebook (in a Docker image) on this [GitHub repo](https://github.com/HamedAlemo/xarray-tutorial).

## Xarray data structures

Like Pandas, xarray has two fundamental data structures:
* a `DataArray`, which holds a single multi-dimensional variable and its coordinates
* a `Dataset`, which holds multiple variables that potentially share the same coordinates

### DataArray

A `DataArray` has four essential attributes:
* `values`: a `numpy.ndarray` holding the array’s values
* `dims`: dimension names for each axis (e.g., `('x', 'y', 'z')`)
* `coords`: a dict-like container of arrays (coordinates) that label each point (e.g., 1-dimensional arrays of numbers, datetime objects or strings)
* `attrs`: an `OrderedDict` to hold arbitrary metadata (attributes)

Let's start by constructing some DataArrays manually 

In [ ]:
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt

In [ ]:
da = xr.DataArray([9, 0, 2, 1, 0])
da

A simple DataArray without dimensions or coordinates isn't much use. We can add a dimension name...

In [ ]:
da = xr.DataArray([9, 0, 2, 1, 0], dims=['x'])
da

But things get more interesting when we add a coordinate:

In [ ]:
da = xr.DataArray([9, 0, 2, 1, 0],
                  dims=['x'],
                  coords={'x': [10, 20, 30, 40, 50]})
da

This coordinate has been used to create an _index_, which works very similar to a `pandas` index.
In fact, under the hood, `xarray`  uses `pandas` index.

In [ ]:
da.indexes

`xarray` has built-in plotting, like `pandas`.

In [ ]:
da.plot(marker='o')

### Multidimensional DataArray

If we are just dealing with 1D data, `pandas` and `xarray` have very similar capabilities. `xarray`'s real potential comes with multidimensional data.

In this example, we use real data from ocean profiling floats. [ARGO](https://www.aoml.noaa.gov/argo/) floats are autonomous robotic instruments that collect *Temperature, Salinity, and Pressure* data from the ocean. ARGO floats collect one “profile” (a set of measurements at different depths or “levels”).

<div>
<center>
<img src="https://argo.ucsd.edu/wp-content/uploads/sites/361/2020/06/float_cycle_1.png" width="800"/>
</div>

Each profile has a single latitude, longitude, and date associated with it, in addition to measurements at different levels.

Let’s start by using [pooch](https://www.fatiando.org/pooch/latest/) to download the data files we need for this exercise. The following code will give you a list of `.npy` files that you can open in the next step.



``` {Note}
Here we are passing a hash to the retrieve method from `pooch`. Hash-based verification ensures that a file has not been corrupted by comparing the file's hash value to a previously calculated value. If these values match, the file is presumed to be unmodified.
```

In [ ]:
import pooch
url = "https://www.ldeo.columbia.edu/~rpa/float_data_4901412.zip"
files = pooch.retrieve(url, processor=pooch.Unzip(), known_hash="2a703c720302c682f1662181d329c9f22f9f10e1539dc2d6082160a469165009")
files.sort()
files

We will manually load each of these variables into a `numpy` array.
If this seems repetitive and inefficient, *that's the point!*
`numpy` itself is not meant for managing groups of inter-related arrays. That's what `xarray` is for!

In [ ]:
P = np.load(files[0])
S = np.load(files[1])
T = np.load(files[2])
date = np.load(files[3])
lat = np.load(files[4])
levels = np.load(files[5])
lon = np.load(files[6])

Let's look at the shape of some of the variables:

In [ ]:
print("Salinity shape is:", S.shape)
print("Levels shape is:", levels.shape)
print("Date shape is:", date.shape)

As expected, Salinity has dimensions of levels x date

Let's organize the data and coordinates of the salinity variable into a DataArray.

In [ ]:
da_salinity = xr.DataArray(S, dims=['level', 'date'],
                           coords={'level': levels,
                                   'date': date},)
da_salinity

In [ ]:
da_salinity.plot()

Since we are working with ocean data, it's helpful to plot the `level` in an increasing order from top to bottom:

In [ ]:
da_salinity.plot(yincrease=False)

Attributes can be used to store metadata. What metadata should you store? The [CF Conventions](http://cfconventions.org/Data/cf-conventions/cf-conventions-1.7/cf-conventions.html#_description_of_the_data) are a great resource for thinking about climate metadata. Below we define two of the required CF-conventions attributes.

In [ ]:
da_salinity.attrs['units'] = 'PSU'
da_salinity.attrs['standard_name'] = 'sea_water_salinity'
da_salinity

Now if we plot the data again, the name and units are automatically attached to the figure.

In [ ]:
da_salinity.plot()

### Datasets

A `Dataset` holds many `DataArray`s which potentially can share coordinates. In analogy to pandas:

    pandas.Series : pandas.Dataframe :: xarray.DataArray : xarray.Dataset
    
Constructing `Dataset`s manually is a bit more involved in terms of syntax. The `Dataset` constructor takes three arguments:

* `data_vars` should be a dictionary with each key as the name of the variable and each value as one of:
  * A `DataArray` or Variable
  * A tuple of the form `(dims, data[, attrs])`, which is converted into arguments for Variable
  * A `pandas` object, which is converted into a `DataArray`
  * A 1D array or list, which is interpreted as values for a one dimensional coordinate variable along the same dimension as it’s name
* `coords` should be a dictionary of the same form as `data_vars`.
* `attrs` should be a dictionary.

Let's put together a Dataset with temperature, salinity and pressure all together

In [ ]:
argo = xr.Dataset(
    data_vars={
        'salinity':    (('level', 'date'), S),
        'temperature': (('level', 'date'), T),
        'pressure':    (('level', 'date'), P)
    },
    coords={
        'level': levels,
        'date': date
    }
)
argo

What about lon and lat? We forgot them in the creation process, but we can add them after the fact.

In [ ]:
argo.coords['lon'] = lon
argo

That was not quite right...we want lon to have dimension `date`:

In [ ]:
del argo['lon']
argo.coords['lon'] = ('date', lon)
argo.coords['lat'] = ('date', lat)
argo

### Coordinates vs. Data Variables

Data variables can be modified through arithmetic operations or other functions. Coordinates are always kept the same.

In [ ]:
argo * 10000

Clearly lon and lat are coordinates and not data variables.

The bold font in the representation above indicates that `level` and `date` are "dimension coordinates" (they describe the coordinates associated with data variable axes) while `lon` and `lat` are "non-dimension coordinates". We can make any variable a non-dimension coordinate (using `set_coords`).

## Selecting Data (Indexing)

We can always use regular `numpy` indexing and slicing on `DataArray`s.

In [ ]:
argo.salinity.shape

In [ ]:
argo.salinity[2]

In [ ]:
argo.salinity[2].plot()

In [ ]:
argo.salinity[:, 10]

In [ ]:
argo.salinity[:, 10:11].plot()

However, it is often more powerful to use `xarray`'s `.sel()` method to use label-based indexing.

In [ ]:
argo.salinity.sel(level=[2, 5])

In [ ]:
argo.salinity.sel(level=2)

Under the hood, this method is powered by using `pandas`’s powerful Index objects which means this method uses `pandas`’s (well documented) logic for indexing. For example, you can use string shortcuts for datetime indexes (e.g., ‘2000-01’ to select all values in January 2000). It also means that slices are treated as inclusive of both the start and stop values, unlike normal Python indexing.


In [ ]:
argo.salinity.sel(level=2).plot()

In [ ]:
argo.salinity.sel(date='2012-10-22')

We can improve the plot by using the `yincrease` argument, and setting the variable that will be plotted on `y` axis:

In [ ]:
argo.salinity.sel(date='2012-10-22').plot(y='level', yincrease=False)

`.sel()` also supports slicing.

In [ ]:
argo.salinity.sel(date=slice('2012-10-01', '2012-12-01'))

In [ ]:
argo.salinity.sel(date=slice('2012-10-01', '2012-12-01')).plot()

`.sel()` also works on the whole Dataset

In [ ]:
argo.sel(date='2012-10-22')

In [ ]:
argo.sel(date = '2012-12-22')

## Computation

`xarray` `DataArrays` and `Datasets` work seamlessly with arithmetic operators and `numpy` array functions.

In [ ]:
temp_kelvin = argo.temperature + 273.15
temp_kelvin.plot(yincrease=False)

We can also combine multiple `xarray` `Dataset`s in arithmetic operations

In [ ]:
g = 9.8
buoyancy = g * (2e-4 * argo.temperature - 7e-4 * argo.salinity)
buoyancy.plot(yincrease=False)

## Broadcasting, Aligment, and Combining Data

### Broadcasting

Broadcasting arrays in `numpy` is a nightmare. It is much easier when the data axes are labeled!

This is a useless calculation, but it illustrates how performing an operation on arrays with different coordinates will result in automatic broadcasting.

In [ ]:
argo.level

In [ ]:
argo.lat

In [ ]:
level_times_lat = argo.level * argo.lat
level_times_lat

In [ ]:
level_times_lat.plot()

### Alignment

If you try to perform operations on `DataArray`s that share a dimension name, `xarray` will try to _align_ them first.
This works nearly identically to `pandas`, except that there can be multiple dimensions to align over.

To see how alignment works, we will create some subsets of our original data.

In [ ]:
sa_surf = argo.salinity.isel(level=slice(0, 20))
sa_mid = argo.salinity.isel(level=slice(10, 30))

By default, when we combine multiple arrays in mathematical operations, `xarray` performs an "inner join".

In [ ]:
(sa_surf * sa_mid).level

We can override this behavior by manually aligning the data:

In [ ]:
sa_surf_outer, sa_mid_outer = xr.align(sa_surf, sa_mid, join='outer')
sa_surf_outer

As we can see, missing data (NaNs) have been filled in where the array was extended. 

From the documentation: Missing values (if `join != 'inner'`) are filled with `fill_value`. The default fill value is NaN.

In [ ]:
sa_surf_outer.plot(yincrease=False)

We can also use `join='right'` and `join='left'`.

If we run our multiplication on the aligned data:

In [ ]:
(sa_surf_outer * sa_mid_outer).level

### Combing Data: Concat and Merge

The ability to combine many smaller arrays into a single big `Dataset` is one of the main advantages of `xarray`.
To take advantage of this, we need to learn two operations that help us combine data:
- `xr.contact`: to concatenate multiple arrays into one bigger array along their dimensions.
- `xr.merge`: to combine multiple different arrays into a `Dataset`.

First let's look at `concat`. Let's re-combine the subset data from the previous step.

In [ ]:
sa_surf_mid = xr.concat([sa_surf, sa_mid], dim='level')
sa_surf_mid

```{warning}
`xarray` will not check the values of the coordinates before `concat`. It will just stick everything together into a new array.
```

In this case, we had overlapping data. We can see this by looking at the `level` coordinate.

In [ ]:
sa_surf_mid.level

In [ ]:
sa_surf_mid.sel(level=11)

In [ ]:
plt.plot(sa_surf_mid.level.values, marker='o')

We can also `concat` data along a _new_ dimension, e.g.

In [ ]:
sa_concat_new = xr.concat([sa_surf, sa_mid], dim='newdim')
sa_concat_new

Note that the data were aligned using an _outer_ join along the non-concat dimensions.

We can merge both `DataArrays` and `Datasets`:

In [ ]:
xr.merge([argo.salinity, argo.temperature])

If the data are not aligned, they will be aligned before merge.
We can specify the join options in `merge`.

In [ ]:
xr.merge([
    argo.salinity.sel(level=slice(0, 30)),
    argo.temperature.sel(level=slice(30, None))
])

In [ ]:
xr.merge([
    argo.salinity.sel(level=slice(0, 30)),
    argo.temperature.sel(level=slice(30, None))
], join='left')

## Reductions

Similar to `numpy`, we can reduce `xarray` `DataArray`s along any number of axes:

In [ ]:
argo.temperature.mean(axis=0)

In [ ]:
argo.temperature.mean(axis=1)

However, instead of performing reductions on axes (as in `numpy`), we can perform them on dimensions. This turns out to be a huge convenience:

In [ ]:
argo_mean = argo.mean(dim='date')
argo_mean

In [ ]:
argo_mean.salinity.plot(y='level', yincrease=False)

In [ ]:
argo_std = argo.std(dim='date')
argo_std.salinity.plot(y='level', yincrease=False)

### Weighted Reductions

Sometimes we want to perform a reduction (e.g. a mean) where we assign different weight factors to each point in the array.
`xarray` supports this via [weighted array reductions](http://xarray.pydata.org/en/stable/user-guide/computation.html#weighted-array-reductions).

As a toy example, imagine we want to weight values in the upper ocean more than the lower ocean.
We could imagine creating a weight array exponentially proportional to pressure as following:

In [ ]:
mean_pressure = argo.pressure.mean(dim='date')
p0 = 250
weights = np.exp(-mean_pressure / p0)
weights.plot()

The weighted mean over the `level` dimensions is calculated as follows:

In [ ]:
temp_weighted_mean = argo.temperature.weighted(weights).mean(dim='level')

Comparing to the unweighted mean, we see the difference:

In [ ]:
temp_weighted_mean.plot(label='weighted')
argo.temperature.mean(dim='level').plot(label='unweighted')
plt.legend()